<a href="https://colab.research.google.com/github/ppiiesle3y1/TLDR/blob/main/AT2_NLP_Pipeline_Pegasus_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.9 MB/s eta 0:00:00


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 92.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00


In [ ]:
pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=e8c936d3a471a9790f856c8d6cd5780403c13e6307c0858b17496213df4fc15b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#load libs
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer,AutoModel
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import nltk
import numpy as np

In [ ]:
#provide huggingface login
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
ds = load_dataset("multi_news")
model_checkpoint="google/pegasus-cnn_dailymail"
metric=load_metric("rouge")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-8-a3885c60a39c>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric("rouge")


## Preprocessing the raw data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_news_length=1024
max_sum_length=128

#tokenize news and target summary
def preprocess(dataset):
    document=[doc for doc in dataset["document"]]
    model_inputs=tokenizer(document,max_length=max_news_length,truncation=True)

    with tokenizer.as_target_tokenizer():
        labels=tokenizer(dataset["summary"],max_length=max_sum_length,truncation=True)
    model_inputs["labels"]=labels["input_ids"]
    return model_inputs
    

In [ ]:
preprocess(ds['train'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[765, 11980, 2657, 108, 126, 123, 116, 120, 166, 435, 108, 3428, 107, 168, 123, 116, 109, 211, 1197, 113, 109, 625, 108, 173, 118, 156, 521, 121, 2770, 121, 56904, 1171, 109, 2656, 113, 2948, 1411, 108, 1741, 111, 4449, 1411, 127, 149, 10237, 124, 156, 474, 151, 8108, 107, 202, 1163, 1597, 124, 109, 475, 107, 283, 107, 2856, 1288, 118, 1268, 4341, 109, 9189, 134, 14087, 114, 107, 208, 107, 351, 859, 166, 1192, 156, 113, 109, 205, 356, 34375, 124, 199, 109, 1968, 53597, 333, 109, 1331, 625, 107, 48208, 127, 118, 599, 17233, 177, 1665, 112, 129, 732, 108, 992, 112, 20061, 42259, 141, 16393, 3852, 77274, 116, 108, 1711, 112, 2168, 18708, 1665, 717, 115, 1538, 107, 139, 9154, 872, 117, 1214, 112, 1137, 6170, 134, 110, 79873, 107, 1063, 134, 2863, 24475, 17548, 108, 145, 123, 267, 129, 1192, 778, 10098, 269, 111, 244, 109, 335, 16244, 109, 9189, 107, 7212, 294, 112, 9835, 121, 386, 681, 108, 879, 109, 1770, 1201, 107, 325, 277, 119, 123, 216, 264, 108, 447, 272, 123, 144, 119

In [ ]:
ds_token = ds.map(preprocess, batched=True)

Map:   0%|          | 0/44972 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 6
args = Seq2SeqTrainingArguments(
    f"TLDR/fined-tuned-pegasus",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    predict_with_generate=True,
    push_to_hub=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size
)

In [ ]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model)
def metrics_eval(pred_eval):
    predictions, labels = pred_eval
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100,labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    #create decoded labels and predictions

    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    #calculate the rouge score for the predictions
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    #calculate length of target summary
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
#create a trainer to train the fine tuned pegasus Model

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=ds_token["train"],
    eval_dataset=ds_token["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=metrics_eval
)

/content/TLDR/fined-tuned-pegasus is already a clone of https://huggingface.co/ppiiesle3y/fined-tuned-pegasus. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.240300,2.091854,41.531600,15.263600,23.857300,36.229000,125.380500


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.240300,2.091854,41.531600,15.263600,23.857300,36.229000,125.380500
2,2.162700,2.074382,41.794000,15.571900,24.124700,36.494200,125.926500


TrainOutput(global_step=14992, training_loss=2.2403732511506136, metrics={'train_runtime': 24788.2425, 'train_samples_per_second': 3.628, 'train_steps_per_second': 0.605, 'total_flos': 2.59885805449003e+17, 'train_loss': 2.2403732511506136, 'epoch': 2.0})